
** Análise Geral de Compras **

* Número total de consumidores
* Preço médio de compra
* Número total de compras
* Rendimento total


** Informações Demográficas Por Gênero **

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados


** Análise de Compras Por Gênero **

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária


** Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela): **

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares


** Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos


** Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra

In [ ]:
# Importação dos pacotes python
import pandas as pd
import matplotlib.pyplot as mlp

In [ ]:
# Carregando o arquivo
arquivo = "dados_compras.json"
base = pd.read_json(arquivo, orient = "records") # oriente usado para 

### Análise Geral de Compras

* Número total de consumidores
* Preço médio de compra
* Número total de compras
* Rendimento total

In [ ]:
# Restringir a base de Dados para facilitar a análise
dados_demo = base.loc[:,["Login","Sexo","Valor"]] #retornar todas as linhas das colunas selecionadas
dados_demo = dados_demo.drop_duplicates("Login")#retira todas os usuário que tiveram mais de uma compra

In [ ]:
total_usuários = dados_demo["Login"].count()# Total de usuários da base demográfica
quantidade_vendas = base["Valor"].count()# Quantidade de vendas realizadas
media_vendas = base["Valor"].mean()# Médidas das vendas realizadas
total_vendas = base["Valor"].sum()# Somatório das vendas realizadas

#criar um dataframe para juntar todas as variáveis criadas 
vendas = pd.DataFrame({"Numero Total de Usuários":total_usuários,
                      "Quantidade de Vendas":quantidade_vendas,
                      "Valor Médio de Vendas": media_vendas,
                      "Total de vendas":total_vendas},
                       index= ["Total"])

#ajustar formato dos numeros com função map
vendas["Numero Total de Usuários"] = vendas["Numero Total de Usuários"].map("{:,}".format)
vendas["Quantidade de Vendas"] = vendas["Quantidade de Vendas"].map("{:}".format)
vendas["Valor Médio de Vendas"] = vendas["Valor Médio de Vendas"].map("R$ {:,.2f}".format)
vendas["Total de vendas"] = vendas["Total de vendas"].map("R$ {:,.2f}".format)

display(vendas)#display apresenta os dados em formato de tabela
#todo - Alinhar texto do dataframe

### Análise Demográfica por Gênero

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados

In [ ]:
# Localizar todas as linhas das colunas sexo e valor agrupa-las por sexo
vendas_sexo = pd.DataFrame(base.loc[:,["Sexo","Valor"]].groupby("Sexo").count())
#realizar a porcentagem da variável anterior
vendas_sexo_por = pd.DataFrame(base.loc[:,["Sexo","Valor"]].groupby("Sexo").count()/total_vendas*100)
#Realizar a média das vendas por gênero
vendas_sexo_me = base.loc[:,["Sexo","Valor"]].groupby("Sexo").mean()

# Concatenar valores em um único Dataframe
novas_vendas = pd.concat(
                        [vendas_sexo,vendas_sexo_por,vendas_sexo_me], 
                        axis= 1, 
                        keys= ["Total de Vendas", "Porcetagem das Vendas", "Media de vendas"])
# axis = 1 indica que o dataframe será concatenado em colunas
# Keys para definir o nome dos títulos

# Formatação dos valores
# Todo ajuste de formatação dos valores
# Todo: testar o assign para criar colunas personalizadas de Porcentagem e Média
# Todo: Retirar Valor e Sexo do Dataframe
# Todo: Alinhar o texto do dataframe

display(novas_vendas)


In [ ]:
#Tipo de plot
fig, ax = mlp.subplots()

#dados do plot
gen = ['Masculino','Feminino','Outro / Não Divulgado']
counts = sexo["Total"]
bar_colors = ['tab:red', 'tab:blue','tab:orange']

#eixo x
ax.bar(gen, counts, color=bar_colors)

#eixo y
ax.set_ylabel('Qntd')

#título
ax.set_title('Qntd por Gênero')

#show
mlp.show()

In [ ]:
#plot das porcentagens

labels = 'Masculino', 'Feminino', 'Outros / Não Divulgado'
sizes = sexo["Total"]
explode = (0.2, 0.1, 0.1)  

fig1, ax1 = mlp.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

mlp.show()

### Análise de Compras Por Gênero

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária

In [ ]:
# Restringir a base de Dados para facilitar a análise
base_genero = base.loc[:,["Sexo","Valor"]]
display(base_genero)

In [ ]:
#agrupar valores por genero com o groupyby
total_gen = base_genero.groupby("Sexo").sum()
total_qntd = base_genero.groupby("Sexo").count()
total_gen_media = base_genero.groupby("Sexo").mean()
total_gen_por = base_genero.groupby("Sexo").sum()/ total_vendas*100

# Concatenar todas as tabelas em um único Dataframe
genero = pd.concat([total_gen,total_qntd,total_gen_media,total_gen_por], 
                   keys= ["Total de Vendas por Gênero", "Quantidade de Vendas", "Media de Vendas", "Porcentagem"], 
                   axis = 1)

#genero.assign()
# Todo: testar o assign para criar colunas personalizadas
# Todo: Retirar Valor e Sexo do Dataframe
# Todo: Alinhar o texto do dataframe

display(genero)

In [ ]:
# Restringir a base de Dados para facilitar a análise
base_idade = base.loc[:,["Idade", "Valor"]]
display(base_idade)

In [ ]:
id1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
id2 = [21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]
id3 = [41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
id4 = [61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]

# Agrupar a soma dos valores de idade
idade= base_idade.groupby("Idade").sum()

display(idade)

#### Consumidores Mais Populares (Top 5)
### Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela): **

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares

In [ ]:
# Restringir a base de Dados para facilitar a análise
base_consumidores = base.loc[:,["Login","Valor","Nome do Item"]]
display(base_consumidores)

In [ ]:
# Agrupar o total de vendas realizado por cada consumidor, ordenandos na ordem decresente do valor total de vendas
top5_tot = pd.DataFrame(base_consumidores.groupby("Login").sum()).sort_values(by = "Valor", ascending= False)
top5_cont = pd.DataFrame(base_consumidores.groupby("Login").count())
top5_med = pd.DataFrame(base_consumidores.groupby("Login").mean())


# Contenar dataframes
top5 = pd.concat([top5_tot,top5_cont,top5_med], axis = 1, keys= ["Valor Total", "Numero de Compras", "Preço Médio de Compra"])

# Todo: Como retornar o nomme do item vendido
# Todo: testar o assign para criar colunas personalizadas
# Todo: Retirar Valor e Sexo do Dataframe
# Todo: Alinhar o texto do dataframe

# Demonstrar apenas os 5 primeiros valores de dataframe
display(top5.head(5))

### Itens Mais Populares
### Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos

In [ ]:
# Restringir a base de Dados para facilitar a análise
base_itens = base.loc[:,["Item ID","Nome do Item", "Valor"]]
display(base_itens)

### Itens Mais Lucrativos
### Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela): **

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra

In [ ]:
# Restringir a base de Dados para facilitar a análise

In [ ]:
# agrupar os items mais lucrativo
item = pd.DataFrame(base_itens.groupby("Nome do Item").sum("Valor")).sort_values(by= "Valor",ascending=False)

# 

display(item.head(5))